In [1]:
!pip install optuna
!pip install transformers
!pip install sentencepiece
!pip install googletrans
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15719 sha256=c7adff582ec75ce219d2ded00a197c1ec577672d41dfdd152dc6fed6721db54e
  Stored in

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import optuna
from sklearn.metrics import accuracy_score, f1_score
from googletrans import Translator
import random

In [3]:
# Cargar el dataset
df = pd.read_csv('youtoxic_english_1000.csv')
df = df.drop(columns=['CommentId', 'VideoId', 'IsSexist', 'IsHomophobic', 'IsRadicalism'])

# Definir columnas tóxicas
toxic_columns = ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative',
                 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsReligiousHate']

# Convertir TRUE/FALSE a 1/0
df[toxic_columns] = df[toxic_columns].replace({'TRUE': 1, 'FALSE': 0})

# Crear una columna 'is_harmful' que sea 1 si alguna de las categorías es 1
df['is_harmful'] = df[toxic_columns].any(axis=1).astype(int)

# Definir X (textos) e y (etiquetas)
X = df['Text']
y = df['is_harmful']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [4]:
import random
import pandas as pd
from nltk.corpus import wordnet
import nltk

# Descargar recursos necesarios de NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')

# Función para reemplazar palabras por sinónimos
def synonym_replacement(text, n=1):
    words = text.split()
    new_words = words.copy()

    for _ in range(n):
        word = random.choice(words)

        # Obtener sinónimos de la palabra
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = random.choice(synonyms).lemmas()[0].name()
            if synonym != word:
                new_words = [synonym if w == word else w for w in new_words]

    return ' '.join(new_words)

# Función para generar datos sintéticos
def generate_synthetic_data(texts, num_synthetic=100, n_synonyms=1):
    synthetic_texts = []

    for _ in range(num_synthetic):
        original_text = random.choice(texts)
        augmented_text = synonym_replacement(original_text, n=n_synonyms)
        synthetic_texts.append(augmented_text)

    return synthetic_texts

# Datos originales (X_train)
# Asegúrate de tener un conjunto de datos, en este caso supongo que `X_train` es un pandas Series con tus textos
# Generar datos sintéticos
synthetic_texts = generate_synthetic_data(X_train.tolist(), num_synthetic=500, n_synonyms=2)

# Convertir a pandas Series
synthetic_texts = pd.Series(synthetic_texts)

# Concatenar los datos sintéticos con los datos originales
X_train = pd.concat([X_train, synthetic_texts], ignore_index=True)
y_train = pd.concat([y_train, y_train.sample(n=len(synthetic_texts), replace=True)], ignore_index=True)

# Verificar el tamaño del nuevo conjunto de datos
print(X_train.shape, y_train.shape)

# Continúa con el entrenamiento del modelo utilizando X_train y y_train


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


(1350,) (1350,)


In [5]:
import optuna
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import BertTokenizerFast
import torch
from sklearn.metrics import accuracy_score

# 1. Tokenización
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Función para preparar los datos para BERT
def preprocess_data(X, y, tokenizer, max_length=128):
    encodings = tokenizer(X.tolist(), truncation=True, padding=True, max_length=max_length)
    dataset = Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': y.tolist()
    })
    return dataset

# Preprocesamos los datos
train_dataset = preprocess_data(X_train, y_train, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
eval_dataset = preprocess_data(X_test, y_test, tokenizer)

In [ ]:
# Función de Optuna para optimizar hiperparámetros
def objective(trial):
    # Hiperparámetros que Optuna optimiza
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 5)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)

    # 2. Definir el modelo
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    # 3. Definir los argumentos de entrenamiento
    training_args = TrainingArguments(
    output_dir='./final_results',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    logging_dir='./final_logs',
    evaluation_strategy="epoch",  # Evaluamos cada época
    save_strategy="epoch",  # Guardamos el modelo cada época
    logging_steps=10,
    save_steps=10,
    load_best_model_at_end=True,
    report_to="none",# Cargamos el mejor modelo al final
)

    # 4. Función de evaluación
    def compute_metrics(p):
        predictions, labels = p
        predictions = predictions.argmax(axis=-1)
        return {'accuracy': accuracy_score(labels, predictions)}

    # 5. Entrenamiento del modelo con Optuna
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=train_dataset,  # Usamos el mismo conjunto para evaluación (podrías separar uno diferente)
        compute_metrics=compute_metrics,
    )

    # 6. Ejecutamos el entrenamiento y obtenemos la métrica de evaluación (accuracy)
    train_results = trainer.train()

    # 7. Optuna maximiza la métrica, en este caso la precisión
    eval_result = trainer.evaluate()

    return eval_result['eval_accuracy']  # Esto es lo que Optuna maximiza


In [ ]:
# Crear un estudio de Optuna y optimizar
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # Ajusta el número de pruebas (trials)

# Imprimir los mejores parámetros encontrados
print(f"Mejores hiperparámetros: {study.best_params}")


[I 2024-11-18 10:06:54,699] A new study created in memory with name: no-name-5aaa52d6-8359-4008-97f2-74ad05a03947
<ipython-input-20-b2b7bd4d249f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
<ipython-input-20-b2b7bd4d249f>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it fo

Epoch,Training Loss,Validation Loss,Accuracy
1,0.698900,0.615108,0.673333
2,0.617300,0.483770,0.784444
3,0.529900,0.393122,0.828148
4,0.412600,0.351214,0.852593
5,0.377600,0.305609,0.854815


[I 2024-11-18 10:12:52,075] Trial 0 finished with value: 0.8548148148148148 and parameters: {'learning_rate': 9.57692272252304e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.0006803695037071061}. Best is trial 0 with value: 0.8548148148148148.
<ipython-input-20-b2b7bd4d249f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
<ipython-input-20-b2b7bd4d249f>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and

Epoch,Training Loss,Validation Loss,Accuracy
1,0.697200,0.672980,0.628889
2,0.674100,0.648647,0.665926
3,0.678000,0.620268,0.691852
4,0.589500,0.606419,0.694074
5,0.615000,0.598474,0.711111


[I 2024-11-18 10:18:19,823] Trial 1 finished with value: 0.7111111111111111 and parameters: {'learning_rate': 3.4853358633521905e-06, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.00016808559126063236}. Best is trial 0 with value: 0.8548148148148148.
<ipython-input-20-b2b7bd4d249f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
<ipython-input-20-b2b7bd4d249f>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased 

...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.698300,0.706988,0.546667
2,0.732000,0.688909,0.546667
3,0.696100,0.693515,0.546667
4,0.692500,0.689165,0.546667
5,0.682900,0.688789,0.546667


[I 2024-11-18 10:46:53,929] Trial 8 finished with value: 0.5466666666666666 and parameters: {'learning_rate': 0.00040797890728965435, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 3.354310956452397e-05}. Best is trial 0 with value: 0.8548148148148148.
<ipython-input-20-b2b7bd4d249f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
<ipython-input-20-b2b7bd4d249f>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased a

Epoch,Training Loss,Validation Loss,Accuracy
1,0.661300,0.587593,0.703704
2,0.525700,0.508174,0.780741
3,0.517300,0.474615,0.798519


[I 2024-11-18 10:50:00,604] Trial 9 finished with value: 0.7985185185185185 and parameters: {'learning_rate': 1.562702310034748e-05, 'batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 4.998165960486738e-06}. Best is trial 0 with value: 0.8548148148148148.


Mejores hiperparámetros: {'learning_rate': 9.57692272252304e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.0006803695037071061}


In [7]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Mejores hiperparámetros encontrados por Optuna
best_params = {
    'learning_rate': 2.2966411075814385e-06,
    'batch_size': 8,
    'num_train_epochs': 5,
    'weight_decay': 0.003001264223004398
}

# Crear el modelo BERT para clasificación binaria (2 clases)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Argumentos de entrenamiento con los mejores hiperparámetros
training_args = TrainingArguments(
    output_dir='./final_results',  # Directorio para los resultados del modelo
    num_train_epochs=best_params['num_train_epochs'],  # Número de épocas
    per_device_train_batch_size=best_params['batch_size'],  # Tamaño de batch para entrenamiento
    per_device_eval_batch_size=best_params['batch_size'],  # Tamaño de batch para evaluación
    learning_rate=best_params['learning_rate'],  # Tasa de aprendizaje
    weight_decay=best_params['weight_decay'],  # Decaimiento del peso
    logging_dir='./final_logs',  # Directorio para los logs
    evaluation_strategy="epoch",  # Evaluar al final de cada época
    logging_steps=10,  # Cada cuántos pasos se loguean los resultados
    save_strategy="epoch",  # Guardar el modelo cada "save_steps" pasos
    save_steps=10,  # Guardar cada 10 pasos
    load_best_model_at_end=True,
    report_to="none",  # Cargar el mejor modelo al final
)

# Función de métrica para calcular la precisión
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Entrenamiento del modelo usando Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Dataset de entrenamiento
    eval_dataset=train_dataset,   # Dataset de evaluación (puedes usar otro conjunto de validación)
    compute_metrics=compute_metrics,
)

# Entrenar el modelo
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.683800,0.658785,0.589630
2,0.629800,0.617579,0.669630
3,0.603000,0.581010,0.717778
4,0.606500,0.561295,0.734815
5,0.596000,0.554944,0.742222


TrainOutput(global_step=845, training_loss=0.6304670226644482, metrics={'train_runtime': 267.6344, 'train_samples_per_second': 25.221, 'train_steps_per_second': 3.157, 'total_flos': 443999905920000.0, 'train_loss': 0.6304670226644482, 'epoch': 5.0})

In [8]:
# Evaluar el modelo en el conjunto de entrenamiento
train_results = trainer.evaluate(train_dataset)

# Evaluar el modelo en el conjunto de prueba
test_results = trainer.evaluate(eval_dataset)

# Extraer las precisiones (accuracy) de los resultados
train_accuracy = train_results['eval_accuracy']
test_accuracy = test_results['eval_accuracy']

# Calcular el porcentaje de overfitting
overfitting_percentage = ((train_accuracy - test_accuracy) / train_accuracy) * 100

# Mostrar el porcentaje de overfitting
print(f"Overfitting Percentage: {overfitting_percentage:.2f}%")


Overfitting Percentage: 6.59%


In [9]:
# Guardar el modelo y el tokenizador
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')